In [ ]:
from scipy.special import lambertw
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline

Rs = 1.
Rsh = 1000.
T0 = 25. + 273.15
Ta = 35 + 273.15
NOCT = 46.
Id0 = 10e-13
Iphi0 = 35e-3 #we suppose a radiation of 1000 W/m²
k = 1.38e-23
q = 1.602e-19
Tc = 298.15
Vt = k*Tc/q
Eg = 1.12
n=1.
G=1000.

V = np.linspace(0,.7, num=100)
I = np.zeros(100, dtype=float)

P = np.zeros(100, dtype=float)
for i in range(100):
    Tc = Ta + G*(NOCT-20.)/800.
    Iphi = Iphi0*G/1000.
    Id = Id0*(T0/Tc)**3 * np.exp(q*Eg/n/k*(1./T0 - 1./Tc))
    temp = lambertw(Id*Rs/(n*Vt*(1.+Rs/Rsh))*np.exp(V[i]/n/Vt*(1.-Rs/(Rs+Rsh))+(Iphi+Id)*Rs/(n*Vt*(1.+Rs/Rsh))), 0)
    I[i] = (Iphi + Id - V[i]/Rsh)/(1.+Rs/Rsh) - n*Vt/Rs*temp.real
    P[i] = I[i]*V[i]

Pm = np.amax(P)
Vm = V[np.where(P==Pm)[0][0]]
temp = lambertw(Id*Rs/(n*Vt*(1.+Rs/Rsh))*np.exp(Vm/n/Vt*(1.-Rs/(Rs+Rsh))+(Iphi+Id)*Rs/(n*Vt*(1.+Rs/Rsh))))
Im = (Iphi + Id - Vm/Rsh)/(1.+Rs/Rsh) - n*Vt/Rs*temp.real
fig, ax = plt.subplots(figsize=(12,8))
ax.plot(V,I,'k-', label='Current')
ax.plot(V,P,'r-', label='Power (W)')
ax.axvline(x=Vm, color='k', linestyle=':')
ax.axhline(y=Im, color='k', linestyle=':')
ax.set_xlim(0,0.7)
ax.set_ylim(0,40e-3)
ax.set_xlabel('Voltage (V)')
ax.set_ylabel('Current (A/cm2)')
ax.grid(True)
#ax.legend()

epsilon=Pm*12*12/(G*12*12/10000)

print('A photocell generates %.2f A and %.2f W' %(Im*12*12, Pm*12*12))
print('the efficiency is', epsilon*100, '%')

In [ ]:
from scipy.special import lambertw
import numpy as np
import matplotlib.pylab as plt
from ipywidgets import interact, FloatSlider, fixed
%matplotlib inline

def PV_cell(G, Ta, Rs , Rsh, Id0, T0, Iphi0, G0, NOCT, n, Eg, Scell, nser, npar):
    q = 1.602e-19
    k = 1.38e-23
    Tcell = 273.15 + Ta + G*(NOCT-20.)/800.
    Vt = k*Tcell/q
    Id = Id0*(T0/Tcell)**3 * np.exp(q*Eg/n/k*(1./T0 - 1./Tcell))
    Iphi = Iphi0*G/G0
    
    V = np.linspace(0,.7*nser, num=100)
    I = np.zeros(100, dtype=float)
    P = np.zeros(100, dtype=float)
    for i in range(100):
        temp = lambertw(Id*Rs/(n*Vt*(1.+Rs/Rsh))*np.exp(V[i]/nser/n/Vt*(1.-Rs/(Rs+Rsh))+(Iphi+Id)*Rs/(n*Vt*(1.+Rs/Rsh))))
        I[i] = npar*Scell*((Iphi + Id - V[i]/nser/Rsh)/(1.+Rs/Rsh) - n*Vt/Rs*temp.real)
        P[i] = V[i]*I[i]

    Pm = np.amax(P)
    Vm = V[np.where(P==Pm)[0][0]]
    temp = lambertw(Id*Rs/(n*Vt*(1.+Rs/Rsh))*np.exp(Vm/nser/n/Vt*(1.-Rs/(Rs+Rsh))+(Iphi+Id)*Rs/(n*Vt*(1.+Rs/Rsh))))
    Im = npar*Scell*((Iphi + Id - Vm/nser/Rsh)/(1.+Rs/Rsh) - n*Vt/Rs*temp.real)
    fig, ax = plt.subplots(figsize=(10,6))
    ax.plot(V,I,'k-', label='Current')
#    ax.plot(V,P,'r-', label='Power (W)')
    ax.axvline(x=Vm, color='k', linestyle=':')
    ax.axhline(y=Im, color='k', linestyle=':')
    ax.set_xlim(0,0.7*nser)
    ax.set_ylim(0,40e-3*Scell*npar)
    ax.text(10,6, 'Pm= %.0f W' %Pm)
    ax.text(10,5.5, 'Ta= %d degC' %Ta)
    ax.text(10,5, 'G= %d W/m2' %G)


    ax.set_xlabel('Voltage (V)')
    ax.set_ylabel('Current (A) / Power (W)')
    ax.grid(True)
    plt.close(fig)
    return fig

interact(PV_cell, G=FloatSlider(min=0, max=1000., value=1000, step=100, continuous_update=False),
         Ta=FloatSlider(min=-20, max=50, value=25, step=5, continuous_update=False),
        Rs=fixed(1.), Rsh=fixed(1000.), Id0=fixed(1e-12), T0=fixed(298.), Iphi0=fixed(35e-3),
         G0=fixed(1000), NOCT=fixed(46), n=fixed(1), Eg=fixed(1.12), Scell=fixed(12*12), nser=fixed(70), npar=fixed(2))
